<a href="https://colab.research.google.com/github/mattscocchia/NHL-Player-Ratings/blob/main/Copy_of_Player_Ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialize

In [ ]:
!pip install cubist
!pip install --upgrade scikit-learn==1.4
#!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 35.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [ ]:
import requests
import numpy as np
import pandas as pd

# 2 minutes 30 seconds to run

seasons=["20142015","20152016","20162017","20172018","20182019","20192020","20202021","20212022","20222023","20232024","20242025"]
teams=["ANA","ARI","BOS","BUF","CGY","CAR","CHI","COL","CBJ","DAL","DET","EDM","FLA","LAK","MIN","MTL","NSH","NJD","NYI","NYR","OTT","PHI","PIT","SJS","SEA","STL","TBL","TOR","UTA","VAN","VGK","WSH","WPG"]

def get_woodmoney_data(payload):
    url = "https://api.puckiq.com/woodmoney"  # Replace with the correct base URL
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        try:
            # Force JSON parsing even if Content-Type isn't set properly
            data = response.json()
        except ValueError:
            # If Content-Type is not application/json, handle raw response text
            data = json.loads(response.text)
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

# Example usage
combined_df2 = pd.DataFrame()
if __name__ == "__main__":
  for season in seasons:
    for team in teams:
        payload = {
            "season": season,
            "team": team
        }
        data = get_woodmoney_data(payload)

        if data:
            # Extract the results
            results = data.get("results", [])

            # Create a DataFrame from the results
            df = pd.DataFrame(results)

            # Print the DataFrame
            #print(df)
            combined_df2 = pd.concat([combined_df2, df], ignore_index=False)

# Load your pivoted dataset (example structure assumed)
# 1. Load the datasets
#woodmoney_old = pd.read_csv('woodmoney_data.csv')
#woodmoney_current = pd.read_csv('woodmoney_data_current.csv')


In [ ]:
# 2. Concatenate the datasets and clean the data
woodmoney = combined_df2.copy()
#woodmoney = pd.concat([woodmoney_old, woodmoney_current], axis=0)
woodmoney.rename(columns={'player_id': 'playerId'}, inplace=True)
woodmoney['season'] = woodmoney['season'].astype(str).str[:4]
woodmoney = woodmoney.groupby(['playerId', 'season', 'name', 'woodmoneytier']).sum().reset_index()

pivoted_woodmoney = woodmoney.pivot(index=['playerId', 'season', 'name'], columns='woodmoneytier', values=['ctoipct','evtoi'])

pivoted_woodmoney.columns = [
    f"ctoipct_{tier}" if col == "ctoipct" else tier
    for col, tier in pivoted_woodmoney.columns
]
pivoted_woodmoney.reset_index(inplace=True)

pivoted_woodmoney['ctoipct_All'] = 100
pivoted_woodmoney['ctoipct_Elite'] = pivoted_woodmoney['Elite']/pivoted_woodmoney['All']
pivoted_woodmoney['ctoipct_Middle'] = pivoted_woodmoney['Middle']/pivoted_woodmoney['All']
pivoted_woodmoney['ctoipct_Gritensity'] = pivoted_woodmoney['Gritensity']/pivoted_woodmoney['All']


# Define weight mapping for each woodmoneytier
tier_weights = {'Elite': 0.9, 'Middle': 0.5, 'Gritensity': 0.2, 'All': 1}

# 1. Apply weights to each column directly
pivoted_woodmoney['Elite_weighted'] = pivoted_woodmoney['Elite'] * tier_weights['Elite']
pivoted_woodmoney['Middle_weighted'] = pivoted_woodmoney['Middle'] * tier_weights['Middle']
pivoted_woodmoney['Gritensity_weighted'] = pivoted_woodmoney['Gritensity'] * tier_weights['Gritensity']
# For 'All', keep it unweighted (raw TOI)
pivoted_woodmoney['All_weighted'] = pivoted_woodmoney['All']

# 2. Calculate total weighted TOI across tiers (excluding 'All')
pivoted_woodmoney['total_weighted_toi'] = (
    pivoted_woodmoney['Elite_weighted'] +
    pivoted_woodmoney['Middle_weighted'] +
    pivoted_woodmoney['Gritensity_weighted']
)

# 3. Calculate the final opponent strength as a weighted average
pivoted_woodmoney['minute_strength'] = (
    pivoted_woodmoney['total_weighted_toi'] /
    1.6
    #pivoted_woodmoney['All']  # Use 'All' as total TOI
)

# 4. Keep only relevant columns for the summary
opponent_strength_summary = pivoted_woodmoney[['playerId', 'name', 'season', 'minute_strength']]

opponent_strength_summary['season'] = pd.Categorical(opponent_strength_summary['season'])
opponent_strength_summary['season'] = opponent_strength_summary['season'].astype(int)

<ipython-input-5-69baa6994693>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opponent_strength_summary['season'] = pd.Categorical(opponent_strength_summary['season'])
<ipython-input-5-69baa6994693>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opponent_strength_summary['season'] = opponent_strength_summary['season'].astype(int)


In [ ]:
# Load the data into a pandas dataframe
df = pd.read_csv('all_players.csv')
cut_year = 2016
data = df[df['season'] >= (cut_year-2)].copy()
data['season'] = pd.Categorical(data['season'])

data = data.dropna()

In [ ]:
games_played_req = 1

new_data = pd.merge(data[(data['situation'] == '5on5') & (data['games_played'] > games_played_req)], opponent_strength_summary[['playerId', 'season', 'minute_strength']], on=['playerId','season'], how='left')

teams = pd.read_csv("all_teams.csv")
new_data['goalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','goalsAgainst','season']], on=['team','season'], how='left')['goalsAgainst']
new_data['goalsFor'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','goalsFor','season']], on=['team','season'], how='left')['goalsFor']
new_data['xGoalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','xGoalsAgainst','season']], on=['team','season'], how='left')['xGoalsAgainst']
new_data['xGoalsFor'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','xGoalsFor','season']], on=['team','season'], how='left')['xGoalsFor']
new_data['team_icetime'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','iceTime','season']], on=['team','season'], how='left')['iceTime']
new_data['pk_team_icetime'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','iceTime','season']], on=['team','season'], how='left')['iceTime']
new_data['pp_team_icetime'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','iceTime','season']], on=['team','season'], how='left')['iceTime']
new_data['pp_goalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','goalsAgainst','season']], on=['team','season'], how='left')['goalsAgainst_y']
new_data['pp_goalsFor'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','goalsFor','season']], on=['team','season'], how='left')['goalsFor_y']
new_data['pp_xgoalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','xGoalsAgainst','season']], on=['team','season'], how='left')['xGoalsAgainst_y']
new_data['pp_xgoalsFor'] = pd.merge(new_data, teams[teams['situation'] == '5on4'][['team','xGoalsFor','season']], on=['team','season'], how='left')['xGoalsFor_y']
new_data['pk_xgoalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','xGoalsAgainst','season']], on=['team','season'], how='left')['xGoalsAgainst_y']
new_data['pk_xgoalsFor'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','xGoalsFor','season']], on=['team','season'], how='left')['xGoalsFor_y']
new_data['pk_goalsAgainst'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','goalsAgainst','season']], on=['team','season'], how='left')['goalsAgainst_y']
new_data['pk_goalsFor'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','goalsFor','season']], on=['team','season'], how='left')['goalsFor_y']
new_data['team_games'] = pd.merge(new_data, teams[teams['situation'] == '4on5'][['team','games_played','season']], on=['team','season'], how='left')['games_played_y']
new_data['xGoalsp'] = pd.merge(new_data, teams[teams['situation'] == '5on5'][['team','xGoalsPercentage','season']], on=['team','season'], how='left')['xGoalsPercentage']

icetime = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['icetime']).reset_index()['icetime']
timeOnBench = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['timeOnBench']).reset_index()['timeOnBench']
oigf = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['OnIce_F_goals']).reset_index()['OnIce_F_goals']
oiga = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['OnIce_A_goals']).reset_index()['OnIce_A_goals']
oixgf = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['OnIce_F_xGoals']).reset_index()['OnIce_F_xGoals']
oixga = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['OnIce_A_xGoals']).reset_index()['OnIce_A_xGoals']
penaltiesT = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['penalties']).reset_index()['penalties']
penaltiesD = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['penaltiesDrawn']).reset_index()['penaltiesDrawn']
takeaways = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_takeaways']).reset_index()['I_F_takeaways']
giveaways = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_giveaways']).reset_index()['I_F_giveaways']
points = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_points']).reset_index()['I_F_points']
I_F_primaryAssists = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_primaryAssists']).reset_index()['I_F_primaryAssists']
I_F_secondaryAssists = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_secondaryAssists']).reset_index()['I_F_secondaryAssists']
I_F_goals = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_goals']).reset_index()['I_F_goals']
I_F_xGoals = data[data['games_played']>games_played_req].pivot(index=['playerId', 'season', 'name'], columns='situation', values=['I_F_xGoals']).reset_index()['I_F_xGoals']

new_data['pk_time'] = icetime['4on5']
new_data['pk_time_bench'] = icetime['4on5']
new_data['pp_time'] = icetime['5on4']
new_data['pk_gf'] = oigf['4on5']
new_data['pp_gf'] = oigf['5on4']
new_data['pk_ga'] = oiga['4on5']
new_data['pp_ga'] = oiga['5on4']
new_data['pk_xgf'] = oixgf['4on5']
new_data['pp_xgf'] = oixgf['5on4']
new_data['pk_xga'] = oixga['4on5']
new_data['pp_xga'] = oixga['5on4']
new_data['penalties'] = penaltiesT['all']
new_data['penaltiesDrawn'] = penaltiesD['all']
new_data['takeaways'] = takeaways['all']
new_data['giveaways'] = giveaways['all']
new_data['pp_points'] = points['5on4']
new_data['I_F_primaryAssists'] = I_F_primaryAssists['all']
new_data['I_F_secondaryAssists'] = I_F_secondaryAssists['all']
new_data['I_F_goals'] = I_F_goals['all']
new_data['I_F_xGoals'] = I_F_xGoals['all']

In [ ]:
from scipy.stats import rankdata
# Combine the two dataframes
combined_df = new_data.copy()


#combined_df[combined_df['season'] == 2024]['icetime'] = combined_df[combined_df['season'] == 2024]['icetime']/combined_df[combined_df['season'] == 2024]['games_played']*82
#combined_df[combined_df['season'] == 2024]['gameScore'] = combined_df[combined_df['season'] == 2024]['gameScore']/combined_df[combined_df['season'] == 2024]['games_played']*82
#combined_df[combined_df['season'] == 2024]['games_played'] = 82

# Step 1: Assign values to 'ford' based on 'position'
def determine_ford(position):
    if "D" in position:
        return "D"
    else:
        return "F"

combined_df["ford"] = combined_df["position"].apply(determine_ford)

# Step 2: Calculate gameScore_percentile by grouping
combined_df["gameScore_percentile"] = (
    combined_df.groupby(["situation", "season", "ford"])["gameScore"]
    .rank(pct=True)
)

# Step 2: Calculate gameScore_percentile by grouping
combined_df["overall_percentile"] = (
    combined_df.groupby(["situation", "season"])["gameScore"]
    .rank(pct=True)
)

combined_df['minute_strength_percentile'] = (
    combined_df.groupby(["situation", "season", "ford"])['minute_strength']
    .rank(pct=True)
)


# Split back the percentiles into the original dataframes
new_data['gameScore_percentile'] = combined_df.loc[:, 'gameScore_percentile'].values

new_data['overall_percentile'] = combined_df.loc[:, 'overall_percentile'].values

new_data['minute_strength_percentile'] = combined_df.loc[:, 'minute_strength_percentile'].values

## Data Prep

In [ ]:
# Calculate each component

### FINAL COPY

new_data['Component_1'] = 0.02*new_data['shotsBlockedByPlayer']
new_data['Component_2'] = 0.015 * (new_data['faceoffsWon'] - new_data['faceoffsLost'])
new_data['Component_3'] = 0.06 * ( - new_data['penalties'])
xga_d = 2*((((new_data['OnIce_A_xGoals'] / new_data['icetime']) -
     (new_data.groupby("season")["xGoalsAgainst"].transform("mean") /
      new_data.groupby("season")["team_icetime"].transform("mean"))))*60*60)
new_data['Component_4'] = np.where(
    new_data['ford'] == 'F',
    1.75 * xga_d,
    2.3 * xga_d
)
#ga_d = 2*(((new_data['OnIce_A_goals']/new_data['icetime']) - new_data['goalsAgainst']/new_data['team_icetime'])*60*60)
ga_d = 2*((((new_data['OnIce_A_goals'] / new_data['icetime']) -
     (new_data.groupby("season")["goalsAgainst"].transform("mean") /
      new_data.groupby("season")["team_icetime"].transform("mean"))))*60*60)
new_data['Component_5'] = np.where(
    new_data['ford'] == 'F',
    0.4375 * ga_d,
    0.575 * ga_d
)
new_data['Component_6'] = new_data['pk_time'] / 60 / 60
new_data['Component_7'] = (0.1*(( new_data['pk_ga']/new_data['pk_time'] - (new_data.groupby("season")["pk_goalsAgainst"].transform("mean")/new_data.groupby("season")["pk_team_icetime"].transform("mean")))*60*60).fillna(0) + 0.1*((new_data['pk_xga']/new_data['pk_time'] - (new_data.groupby("season")["pk_xgoalsAgainst"].transform("mean")/new_data.groupby("season")["pk_team_icetime"].transform("mean")))*60*60).fillna(0))
new_data['pk_rating'] = np.where(
    new_data['Component_6'] > 0.3,
    -2*((0.5*new_data['Component_7']))*new_data['Component_6'],
    0
)
base_rating = -2*(new_data['Component_4'] + new_data['Component_5'])-(new_data['Component_4'] - new_data['Component_5'])*0.1

# Apply condition: if base_rating < 0, multiply by (1 - minute_strength_percentile), otherwise multiply by minute_strength_percentile
new_data['5on5_drating'] = np.where(
    base_rating < 0,
    base_rating * (1 - new_data['minute_strength_percentile']),
    base_rating * new_data['minute_strength_percentile']
)


# Step 2: Calculate gameScore_percentile by grouping
new_data["5on5_drating_pct"] = (
    new_data.groupby(["season"])["5on5_drating"]
    .rank(pct=True)
)

def_rating = (
    new_data['Component_1'] +
    new_data['Component_2'] +
    new_data['Component_3'] +
    new_data['5on5_drating'] +
    new_data['pk_rating'] +
    0.01 * new_data['I_F_takeaways']
) * (new_data['games_played']/new_data['team_games'])

new_data['def_rating'] = np.where(
    new_data['ford'] == 'F',
    def_rating-8,
    def_rating-0.5
)

# Step 2: Calculate gameScore_percentile by grouping
new_data["def_percentile"] = (
    new_data.groupby(["season","ford"])["def_rating"]
    .rank(pct=True)
)

# Display the components and the result
components_table = new_data[['name','season', 'games_played', 'team', 'ford','minute_strength_percentile', 'Component_1', 'Component_2', 'Component_3', 'Component_4',
                             'Component_5', 'Component_6', 'Component_7','pk_rating','5on5_drating', '5on5_drating_pct', 'def_rating', 'def_percentile','I_F_takeaways']]#[new_data['season'] == 2023]

components_table

,name,season,games_played,team,ford,minute_strength_percentile,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,pk_rating,5on5_drating,5on5_drating_pct,def_rating,def_percentile,I_F_takeaways
0,Jaromir Jagr,2014,77,FLA,F,0.834234,0.24,0.000,-1.44,-1.311535,-0.241289,0.026389,-1.252421,0.0,2.680121,0.951537,1.718528,0.888288,35
1,Jaromir Jagr,2015,79,FLA,F,0.910995,0.32,-0.015,-1.38,0.309945,0.067640,0.006111,-1.233907,0.0,-0.069371,0.453179,-0.794211,0.171030,32
2,Jaromir Jagr,2016,81,FLA,F,0.917399,0.08,0.000,-1.68,0.078938,0.010437,0.015556,-1.306862,0.0,-0.015331,0.478972,-1.230144,0.086116,37
3,Jaromir Jagr,2017,22,CGY,F,0.249117,0.16,0.000,-0.30,-0.588153,-0.817859,0.000000,0.000000,0.0,0.694800,0.729070,0.175678,0.586572,10
4,Sergei Gonchar,2014,48,MTL,D,0.384880,1.16,0.015,-0.54,-0.557652,-0.992405,0.280278,-0.185622,0.0,1.176438,0.795508,0.583769,0.680412,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9616,Juha Jaaska,2024,15,CAR,F,0.170686,0.06,0.030,-0.18,-4.304509,-1.263030,0.028889,-1.386228,0.0,1.952521,0.894673,0.520145,0.679035,1
9617,Arttu Hyry,2024,5,DAL,F,0.089054,0.06,0.090,0.00,3.969834,-0.904259,0.000000,0.000000,0.0,-6.029152,0.047215,-0.553694,0.263451,1
9618,Maxim Tsyplakov,2024,49,NYI,F,0.810761,0.42,0.015,-0.90,0.142721,-0.220894,0.003333,-1.455459,0.0,0.097279,0.500000,-0.182799,0.374768,17
9619,Jere Innala,2024,15,COL,F,0.137291,0.10,-0.030,-0.06,-2.508810,0.028571,0.009167,-1.455459,0.0,0.715866,0.696126,0.200691,0.584416,1


In [ ]:

# Calculate each component
new_data['Component_1'] = np.where(
    new_data['ford'] == 'F',
    0.3 * new_data['I_F_goals'],
    0.4 * new_data['I_F_goals']
)

new_data['Component_2'] = 0.4 * new_data['I_F_primaryAssists']
new_data['Component_3'] = 0.25 * new_data['I_F_secondaryAssists']

new_data['Component_8'] = np.where(
    new_data['ford'] == 'F',
    0.25 * new_data['I_F_xGoals'],
    0.15 * new_data['I_F_xGoals']
)

# Team xGoalsFor - Individual xGoalsFor = offIceXGoalsFor
xgf_o = 2*((((new_data['OnIce_F_xGoals'] / new_data['icetime']) -
     (new_data.groupby("season")["xGoalsFor"].transform("mean") /
      new_data.groupby("season")["team_icetime"].transform("mean"))))*60*60)
new_data['Component_4'] = np.where(
    new_data['ford'] == 'F',
    0.625 * xgf_o,
    1.7 * xgf_o
)
gf_o = 2*((((new_data['OnIce_F_goals'] / new_data['icetime']) -
     (new_data.groupby("season")["goalsFor"].transform("mean") /
      new_data.groupby("season")["team_icetime"].transform("mean"))))*60*60)
new_data['Component_5'] = np.where(
    new_data['ford'] == 'F',
    0.625 * gf_o,
    0.425 * gf_o
)
new_data['Component_6'] = new_data['pp_time'] / 60 / 60
new_data['Component_7'] = (0.1*((new_data['pp_gf']/new_data['pp_time'] - (new_data.groupby("season")["pp_goalsFor"].transform("mean")/new_data.groupby("season")["pp_team_icetime"].transform("mean")))*60*60).fillna(0) + 0.1*(((-new_data['pp_xgf']/new_data['pp_time'] - (new_data.groupby("season")["pp_xgoalsFor"].transform("mean")/new_data.groupby("season")["pp_team_icetime"].transform("mean"))))*60*60).fillna(0))
#new_data['Component_7'] = (0.15*((new_data['pp_gf']/new_data['pp_time'] - (new_data['pp_goalsFor']/new_data['pp_team_icetime']))*60*60).fillna(0) + 0.05*(((new_data['pp_xgf']/new_data['pp_time'] - new_data['pp_xgoalsFor']/new_data['pp_team_icetime']))*60*60).fillna(0))
#new_data['pp_rating'] = -10*((0.5*new_data['Component_7']))*new_data['Component_6']/(new_data['pp_team_icetime']/60/60)
new_data['pp_rating'] = np.where(
    new_data['Component_6'] > 0.3,
    -2*((0.5*new_data['Component_7']))*new_data['Component_6'],
    0
)
base_rating = (new_data['Component_4'] + new_data['Component_5'])-(new_data['Component_4'] - new_data['Component_5'])*0.1

# Apply condition: if base_rating < 0, multiply by (1 - minute_strength_percentile), otherwise multiply by minute_strength_percentile
new_data['5on5_orating'] = np.where(
    base_rating < 0,
    base_rating * (1 - new_data['minute_strength_percentile']),
    base_rating * new_data['minute_strength_percentile']
)


# Step 2: Calculate gameScore_percentile by grouping
new_data["5on5_orating_pct"] = (
    new_data.groupby(["season"])["5on5_orating"]
    .rank(pct=True)
)

off_rating = (
    new_data['Component_1'] +
    new_data['Component_2'] +
    new_data['Component_3'] +
    new_data['Component_8'] +
    new_data['5on5_orating'] +
    new_data['pp_rating'] +
    0.06 * (new_data['penaltiesDrawn']) -
    0.01 * new_data['giveaways']
) * (new_data['games_played']/new_data['team_games']) /1.5

new_data['off_rating'] = np.where(
    new_data['ford'] == 'F',
    off_rating-8,
    off_rating-4
)

# Step 2: Calculate gameScore_percentile by grouping
new_data["off_percentile"] = (
    new_data.groupby(["season","ford"])["off_rating"]
    .rank(pct=True)
)

# Display the components and the result
components_table = new_data[['name','season', 'games_played', 'team', 'ford','minute_strength_percentile', 'Component_1', 'Component_2', 'Component_3', 'Component_4',
                             'Component_5', 'Component_6', 'Component_7','Component_8','pp_rating','5on5_orating', '5on5_orating_pct', 'off_rating', 'off_percentile']]#[new_data['season'] == 2021]

components_table

,name,season,games_played,team,ford,minute_strength_percentile,Component_1,Component_2,Component_3,Component_4,Component_5,Component_6,Component_7,Component_8,pp_rating,5on5_orating,5on5_orating_pct,off_rating,off_percentile
0,Jaromir Jagr,2014,77,FLA,F,0.834234,5.1,8.4,2.25,-0.030732,0.218631,2.828889,-1.054739,5.1700,2.983738,0.177554,0.689125,7.463411,0.830631
1,Jaromir Jagr,2015,79,FLA,F,0.910995,8.1,8.4,4.50,0.378652,1.876547,3.424722,-1.283518,4.9400,4.395694,2.190932,0.996532,13.070922,0.958115
2,Jaromir Jagr,2016,81,FLA,F,0.917399,4.8,8.0,2.50,0.382330,0.758955,3.476944,-1.407871,5.7600,4.895089,1.081565,0.950935,10.351212,0.903339
3,Jaromir Jagr,2017,22,CGY,F,0.249117,0.3,1.6,0.50,0.620274,0.646899,0.553889,-1.507901,0.7450,0.835210,0.316337,0.770930,-7.242244,0.282686
4,Sergei Gonchar,2014,48,MTL,D,0.384880,0.4,2.4,1.75,-0.616843,-0.285926,1.529167,-1.256726,0.2580,1.921743,-0.534956,0.293144,-1.695693,0.481100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9616,Juha Jaaska,2024,15,CAR,F,0.170686,0.0,0.8,0.25,0.093093,-0.648778,0.025556,-1.533317,0.4625,0.000000,-0.522361,0.315981,-7.814789,0.217069
9617,Arttu Hyry,2024,5,DAL,F,0.089054,0.0,0.4,0.00,-0.683397,-1.283621,0.000000,0.000000,0.0375,0.000000,-1.846525,0.095642,-8.085473,0.118738
9618,Maxim Tsyplakov,2024,49,NYI,F,0.810761,2.1,2.8,2.25,0.559820,0.461488,1.361389,-1.420533,2.7550,1.933898,0.820064,0.869249,-0.265420,0.662338
9619,Jere Innala,2024,15,COL,F,0.137291,0.0,0.0,0.00,-1.412811,-2.195396,0.031111,-1.615771,0.2850,0.000000,-3.180346,0.043584,-8.508245,0.009276


In [ ]:
new_data['ovr_rating'] = new_data['off_rating'] + new_data['def_rating']

# Step 2: Calculate gameScore_percentile by grouping
new_data["ovr_ptile"] = (
    new_data.groupby(["season","ford"])["ovr_rating"]
    .rank(pct=True)
)

new_data['ovr_pg'] = new_data['ovr_rating']/new_data['games_played']

new_data['ovr_rating_pr'] = new_data['ovr_pg']*82

# Step 2: Calculate gameScore_percentile by grouping
new_data["ovr_ptile_pr"] = (
    new_data.groupby(["season","ford"])["ovr_rating_pr"]
    .rank(pct=True)
)

# Step 3: Calculate Percentiles for Summed Data
components_table = new_data[['name','season', 'ford', 'games_played', 'team', 'off_rating', 'def_rating', 'off_percentile', 'def_percentile', 'ovr_rating', 'ovr_ptile','ovr_pg', 'ovr_rating_pr', 'ovr_ptile_pr']]

components_table

,name,season,ford,games_played,team,off_rating,def_rating,off_percentile,def_percentile,ovr_rating,ovr_ptile,ovr_pg,ovr_rating_pr,ovr_ptile_pr
0,Jaromir Jagr,2014,F,77,FLA,7.463411,1.718528,0.830631,0.888288,9.181939,0.855856,0.119246,9.778169,0.854054
1,Jaromir Jagr,2015,F,79,FLA,13.070922,-0.794211,0.958115,0.171030,12.276711,0.926702,0.155401,12.742916,0.933682
2,Jaromir Jagr,2016,F,81,FLA,10.351212,-1.230144,0.903339,0.086116,9.121068,0.854130,0.112606,9.233674,0.843585
3,Jaromir Jagr,2017,F,22,CGY,-7.242244,0.175678,0.282686,0.586572,-7.066566,0.300353,-0.321208,-26.339018,0.234982
4,Sergei Gonchar,2014,D,48,MTL,-1.695693,0.583769,0.481100,0.680412,-1.111924,0.539519,-0.023165,-1.899537,0.518900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9616,Juha Jaaska,2024,F,15,CAR,-7.814789,0.520145,0.217069,0.679035,-7.294644,0.307978,-0.486310,-39.877390,0.196660
9617,Arttu Hyry,2024,F,5,DAL,-8.085473,-0.553694,0.118738,0.263451,-8.639167,0.074212,-1.727833,-141.682336,0.079777
9618,Maxim Tsyplakov,2024,F,49,NYI,-0.265420,-0.182799,0.662338,0.374768,-0.448219,0.630798,-0.009147,-0.750080,0.627087
9619,Jere Innala,2024,F,15,COL,-8.508245,0.200691,0.009276,0.584416,-8.307554,0.105751,-0.553837,-45.414628,0.183673


In [ ]:
# Step 1: Create overall rating
new_data['ovr_rating'] = new_data['off_rating'] + new_data['def_rating']

# Step 2: Aggregate over multiple seasons
summed_data = (
    new_data[(new_data['season'] > 2021) & (new_data['season'] <= 2024)]
    .groupby(["playerId","ford","name"], as_index=False)  # Group by player and team/ford
    .agg({
        "games_played": "sum",
        "off_rating": "sum",
        "def_rating": "sum",
        "ovr_rating": "sum"
    })
)

summed_data['ovr_pg'] = summed_data['ovr_rating']/summed_data['games_played']

# Step 3: Calculate Percentiles for Summed Data
summed_data["ovr_ptile"] = summed_data.groupby("ford")["ovr_rating"].rank(pct=True)

# Step 4: Create the Final Table
components_table = summed_data[['name', 'ford', 'games_played', 'off_rating', 'def_rating', 'ovr_rating', 'ovr_ptile', 'ovr_pg']]

# Display the table
components_table


,name,ford,games_played,off_rating,def_rating,ovr_rating,ovr_ptile,ovr_pg
0,Eric Staal,F,72,-0.480541,-0.041898,-0.522439,0.665765,-0.007256
1,Ryan Suter,D,218,1.959443,7.527253,9.486696,0.803109,0.043517
2,Jeff Carter,F,151,-0.171759,3.021532,2.849772,0.707713,0.018873
3,Zach Parise,F,112,-0.731386,2.035499,1.304113,0.692828,0.011644
4,Brent Burns,D,218,30.895309,12.977415,43.872723,0.981865,0.201251
...,...,...,...,...,...,...,...,...
1120,Juha Jaaska,F,15,-7.814789,0.520145,-7.294644,0.535859,-0.486310
1121,Arttu Hyry,F,5,-8.085473,-0.553694,-8.639167,0.368065,-1.727833
1122,Maxim Tsyplakov,F,49,-0.265420,-0.182799,-0.448219,0.667118,-0.009147
1123,Jere Innala,F,15,-8.508245,0.200691,-8.307554,0.411367,-0.553837


In [ ]:
# Step 1: Create overall rating
new_data['ovr_rating'] = new_data['off_rating'] + new_data['def_rating']

# Step 2: Aggregate over multiple seasons
summed_data = (
    new_data#[(new_data['season'] > 2021) & (new_data['season'] <= 2024)]
    .groupby(["team","season"], as_index=False)  # Group by player and team/ford
    .agg({
        "games_played": "sum",
        "off_rating": "sum",
        "def_rating": "sum",
        "ovr_rating": "sum",

    })
)

summed_data['ovr_pg'] = summed_data['ovr_rating']/summed_data['games_played']

# Step 3: Calculate Percentiles for Summed Data
summed_data["ovr_ptile"] = summed_data.groupby("season")["ovr_rating"].rank(pct=True)
summed_data["ovr"] = summed_data.groupby("season")["ovr_rating"].rank(ascending=False)

# Step 4: Create the Final Table
components_table = summed_data[['team','season', 'games_played', 'off_rating', 'def_rating', 'ovr_rating', 'ovr_ptile', 'ovr', 'ovr_pg']]

# Display the table
components_table
